In [ ]:
import json
import pandas as pd
import Levenshtein as lv
import os
import pickle
import matplotlib.pyplot as plt

In [ ]:
def measure_time_distance(timedelta):
    """
    Function that convert timedelta into seconds
    """
    distance = (timedelta.total_seconds())
    return distance

def measure_text_distance(x,y):
    """
    Function that calculate the Levenshtein distance
    between two submissions code.
    https://en.wikipedia.org/wiki/Levenshtein_distance
    """
    return lv.distance(x,y)

def calculate_distribution(df, column_name, exercise, mean=True):
    """
    df: Dataframe with submissions
    column_name: column with status

    Function to obtain distribution of submissions status
    return metrics and amount of submissions
    """
    total_amount_submissions = df.shape[0]
    submissions_grouped = df.groupby([column_name]).size()
    metrics = {}
    if mean:
        metrics = submissions_grouped/total_amount_submissions
    else:
        metrics = submissions_grouped
    metrics['exercise'] = str(exercise)
    metrics['submission_amount'] = total_amount_submissions
    return metrics

def shift_columns(df, shift_exercise=False):
    """
    Function that create 4 new columns
    shifted column contains submissions_content shifted to calculate the distance
    datetime_shifted contains datetime shifted to calculate the distance between times
    
    distance, Levenshtein distance between submissions
    time_dist, time distance between submissions in seconds
    """
    df.loc[:,'shifted'] = df['content'].shift().fillna(value='')
    df.loc[:,'datetime_shifted'] = df['datetime'].shift().fillna(value=pd.Timestamp(1800, 1, 1, 0))
    df['distance'] = df.apply(lambda x : measure_text_distance(x['content'],x['shifted']), axis=1)
    df['time_dist'] = df.apply(lambda x : measure_time_distance(x['datetime']-x['datetime_shifted']), axis=1)
    if shift_exercise:
        df.loc[:,'exercise.name.previous'] = df['exercise.name'].shift().fillna(value='')
        df.loc[:,'submissions_status.previous'] = df['status'].shift().fillna(value='')
        df['switch_exercise'] = df.apply(lambda x : x['exercise.name'] != x['exercise.name.previous'], axis=1)
    return df

# Load Failed Submissions

In [ ]:
submissions_failed = pd.DataFrame()
dataset_home = "datasets/"
files = [
    'haskell_new.json'
]

for file in files:
    json_file = open(dataset_home+str(file), encoding='utf8')
    json_data = json.load(json_file)
    submissions_failed = pd.io.json.json_normalize(json_data)
    submissions_failed['datetime'] = pd.to_datetime(submissions_failed['created_at.$date'], unit='ms')

In [ ]:

submissions_failed = submissions_failed[['content', 'expectation_results', 'feedback', 'result', 'sid', 'status',
       'submissions_count', 'test_results', 'guide.slug', 'submitter.email',
       'submitter.social_id', 'guide.name', 'exercise.name', 'exercise.eid',
       'guide.language.name', 'datetime']]


# Load IntroAlgo Submissions

In [ ]:
submissions_introAlgo = pd.DataFrame()
dataset_home = "datasets/"
files = [
    #'2016-2c-introalgo-2c-2016.json', '2017-2c-introalgo-2c-2017.json',
    '2018-2c-introalgo-2018-2c-23Octubre.json',
]

for file in files:
    json_file = open(dataset_home+str(file), encoding='utf8')
    json_data = json.load(json_file)
    
    subm_df = pd.io.json.json_normalize(
        json_data, 'submissions', [['guide','slug'], ['student','email'],['student','social_id'],['guide','name'], ['exercise','name'],['exercise','eid'],['guide','language','name'], ['created_at']], record_prefix='submissions_', errors='ignore')
    subm_df['datetime'] = pd.to_datetime(subm_df['submissions_created_at'])
    submissions_introAlgo = submissions_introAlgo.append(subm_df)
    

In [ ]:
submissions_introAlgo.rename(columns={'submissions_content':'content',
       'submissions_expectation_results': 'expectation_results', 'submissions_feedback':'feedback',
       'submissions_result' : 'result', 'submissions_sid' : 'sid', 'submissions_status': 'status',
       'submissions_submissions_count':'submissions_count', 'submissions_test_results':'test_results',
       'student.email' : 'submitter.email', 'student.social_id':'submitter.social_id'}, inplace=True)

In [ ]:
submissions_introAlgo = submissions_introAlgo.reset_index()[['content', 'expectation_results', 'feedback',
       'result', 'sid', 'status', 'submissions_count', 'test_results',
       'guide.slug', 'submitter.email', 'submitter.social_id', 'guide.name',
       'exercise.name', 'exercise.eid', 'guide.language.name', 'datetime']]

In [ ]:
submissions_introAlgo

# Load courses submissions

In [ ]:
dataset_home = "datasets/haskell"
files = os.listdir(dataset_home)
submissions_courses = pd.DataFrame()
for file in files:
    json_file = open(os.path.join(dataset_home, file), encoding='utf8')
    json_data = json.load(json_file)
    submissions_courses = submissions_courses.append(pd.io.json.json_normalize(json_data), ignore_index=True)

submissions_courses['datetime'] = pd.to_datetime(submissions_courses['created_at.$date'], unit='ms')
submissions_courses = submissions_courses[['content', 'expectation_results', 'feedback', 'result', 'sid', 'status',
   'submissions_count', 'test_results', 'guide.slug', 'submitter.email',
   'submitter.social_id', 'guide.name', 'exercise.name', 'exercise.eid',
   'guide.language.name', 'datetime']]


In [ ]:
submissions_df = pd.concat([submissions_introAlgo, submissions_courses, submissions_failed])

In [ ]:
#base = 'dataframes_pkl/'
#submissions = pd.read_pickle(base + 'failed_submissions_df_FINAL.pkl')
submissions_df =  pd.read_pickle('dataframes_pkl/introAlgo_FINAL_anotado.pkl')
submissions_df

## Users logged in with social accounts.

In [ ]:
social_accounts = submissions_df[submissions_df['submitter.email'].isnull()]['submitter.social_id'].nunique()

## Users logged in with email accounts.

In [ ]:
email_accounts = submissions_df['student.email'].nunique()

In [ ]:
exercises_social = 0
for social in social_accounts:
    exercises_social += submissions_df[submissions_df['student.email'] == social]['exercise.name'].nunique()

In [ ]:
exercises_email = 0
for email in email_accounts:
    exercises_email += submissions_df[submissions_df['submitter.email'] == email]['exercise.name'].nunique()

In [ ]:
submissions_df = submissions_df[submissions_df['guide.language.name'] == 'haskell']
submissions_df = submissions_df[~submissions_df['content'].isnull()]
submissions_df = submissions_df[~(submissions_df['status']=='aborted')]
exercises_names_2018 = ['Guardas: Básico', 'Otherwise', 'La historia sin fin',
       'No todo lo que brilla es Oro', 'Iterar', 'Variable Anónima',
       '¡Terminala! Parte 2', 'El caso base',
       'Pattern Matching con Tuplas', 'Orden de las Condiciones',
       'Cuando los patrones no encajan', 'esMultiploDeTres',
       'Pausa: Volvieron los tipos', 'esMultiploDe', 'Cubo',
       'Puntos Para Setenta!', 'Área', 'Tipando tuplas', 'esBisiesto',
       'celsiusToFahr', 'fahrToCelsius', 'haceFrioF', 'Los números',
       'fibonacci', 'pertenece', 'interseccion', 'transformadaLoca',
       'productoria', 'maximo', 'menoresA', 'diferencias', 'Dispersión',
       'sinRepetidos', 'Pinos', 'esCuadradoPerfecto', 'Días',
       'Valores y variables', 'siguiente', 'mitad', 'Inversa', 'Triple',
       'esNumeroPositivo', 'Más valores', 'Las Funciones',
       'elementos3-upla', 'aplicar2-upla', 'cuentaBizarra', 'notas',
       'notasFinales2-upla', 'esMayorDeEdad', 'calcular', 'Más funciones',
       'Los booleanos', 'Múltiples parámetros', 'Triángulos',
       'Combinando funciones', 'Composición', 'promedios',
       'promediosSinAplazos', 'Los operadores son funciones',
       'Más composición', 'alVesre', 'zipWith', 'filtrar', 'maximoSegun',
       'aplanar', 'intercalar']
#calculate_distribution()

In [ ]:
submissions = pd.DataFrame()
for exercise_name in exercises_names_2018:
    submissions = submissions.append(submissions_df[submissions_df['exercise.name'] == exercise_name], ignore_index = True)

In [ ]:
calculate_distribution(submissions, 'submissions_status', 'todes', False)

In [ ]:
print("total users {}".format(social_accounts + email_accounts))

In [ ]:
submissions = submissions[~(submissions['status'] == 'aborted')]
submissions = submissions[~(submissions['status'] == 'pending')]
submissions = submissions[~(submissions['status'] == 'running')]
distribution = calculate_distribution(submissions, 'status', 'Exercises Distribution', False)
distribution

In [ ]:
submissions_df['exercise.name'].nunique()

In [ ]:
base = 'dataframes_pkl/'
#submissions_df = pd.read_pickle(base+'failed_submissions_df.pkl')
submissions_df = pd.read_pickle('dataframes_pkl/introAlgo_FINAL_anotado.pkl')
#submissions_df = pd.read_pickle('dataframes_pkl/mumuki_io_FINAL_anotado.pkl')


# Calculo de uso de la plataforma

In [ ]:
def calculate_day_and_hour_of_use(df):
    df['day_of_week'] = df['datetime'].apply(
        lambda x : x.isoweekday()
    )
    df['hour_of_day'] = df['datetime'].apply(
        lambda x : x.hour
    )
    return df

In [ ]:
submissions = calculate_day_and_hour_of_use(submissions_df)
submissions

In [ ]:
submissions_per_day = submissions[
    ['datetime', 'day_of_week']].groupby('day_of_week').count().reset_index().rename(columns={'datetime':'amount_submissions'})

## soluciones por dia 1 Lunes 7 Domingo

In [ ]:
submissions_per_day['percentage'] = submissions_per_day['amount_submissions']/submissions_per_day['amount_submissions'].sum()*100
submissions_per_day

## soluciones por franja horaria martes y jueves

In [ ]:
submissions_introAlgo = submissions
submissions_per_hour = submissions_introAlgo[
    (submissions_introAlgo['day_of_week'] == 4) | (submissions_introAlgo['day_of_week'] == 2)][['hour_of_day', 'datetime']].groupby('hour_of_day').count()

In [ ]:
submissions_per_hour = submissions[['hour_of_day', 'datetime']].groupby('hour_of_day').count()

In [ ]:
submissions_per_hour.sum()

In [ ]:
submissions_per_hour = submissions_per_hour.reset_index().rename(columns={'datetime':'amount_submissions'})
submissions_per_hour.sort_values('hour_of_day', inplace=True)
submissions_per_hour.set_index('hour_of_day')
submissions_per_hour

In [ ]:
print(submissions_per_hour.loc[0:3,:].sum())
print('-----------------')
print(submissions_per_hour.loc[0:3,:]['amount_submissions'].sum())

In [ ]:
sub_0_3 = submissions_per_hour.loc[0:3,:]['amount_submissions'].sum()
sub_4_8 = submissions_per_hour.loc[4:8,:]['amount_submissions'].sum()
sub_9_13 = submissions_per_hour.loc[9:13,:]['amount_submissions'].sum()
sub_14_18 = submissions_per_hour.loc[14:18,:]['amount_submissions'].sum()
sub_19_23 = submissions_per_hour.loc[19:23,:]['amount_submissions'].sum()
lst_uso = [sub_0_3, sub_4_8, sub_9_13, sub_14_18, sub_19_23]
lst_uso

In [ ]:
def plot_uso_por_horas(lst_uso):
    plt.gcf().clear()
    plt.style.use('seaborn')
    plt.figure(figsize=(15,10))
    plt.title("Intro Algo", size=30)
    plt.bar(
        ['0-3', '4-8', '9-13', '14-18', '19-23'],
        lst_uso, edgecolor='black', label='Cantidad de soluciones', width=0.5)
    plt.xlabel('Franja Horaria', size=24)
    #plt.ylabel('Cantidad de soluciones enviadas', size=24)
    plt.yticks(fontsize = 25)
    plt.xticks(fontsize = 25)
    plt.legend(loc=2, prop={'size': 20})
    plt.show()


In [ ]:
def plot_uso_semanal(df):
    plt.gcf().clear()
    plt.style.use('seaborn')
    plt.figure(figsize=(15,10))
    plt.title("Intro Algo", size=30)
    plt.bar(
        ['L', 'M', 'X', 'J', 'V', 'S', 'D'],
        df['amount_submissions'], edgecolor='black', label='Cantidad de soluciones', width=0.5)
        #(df['amount_submissions']/df['amount_submissions'].sum())*100, edgecolor='black', label='Submissions Amount')
    plt.xlabel('Dia de la semana', size=24)
    #plt.ylabel('Cantidad de soluciones enviadas', size=24)
    plt.yticks(fontsize = 25)
    plt.xticks(fontsize = 25)
    plt.legend(loc=1, prop={'size': 20})
    plt.show()
    

In [ ]:
plot_uso_semanal(submissions_per_day)

In [ ]:
plot_uso_por_horas(lst_uso)